# Importing Packages

In [2]:
"""
Importing (using/include) packages and files needed for the code to run

Note: that we import pyplot last to avoid a name conflict with grid
"""

using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using DataFrames
using PyCall
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("Temperature.jl")
include("GridOperations.jl")
using PyPlot
include("Visualization.jl")

plots (generic function with 1 method)

# Model Topography Setup

In [3]:
function initial_ice_depth(x::Float64)
    """
    Arguments:
    x::Float64 -- A horizontal value of float64

    Returns:
    The vertical value (depth) corresponding to the horizontal argument value

    Info:
    The initial topography setup follows the following.
    y = A*sin[k(x-b)]+c
    |A| is the amplitude
    k is the wave number (k=2π/wavelength)
    The period is 2π/k
    The horizontal shift is b
    The vertical shift is c
    """
    w = 90000.0
    A = 0.1*90000.0
    k = (2*pi)/w
    b = 0.0
    c = 0.5*90000.0
    return (A)*(sin(k*(x-b)))+(c)
end

function initial_surface_depth(x::Float64)
    """
    Arguments:
    x::Float64 -- A horizontal value of float64

    Returns:
    The vertical value (depth) corresponding to the horizontal argument value
    """
     return 1e4
end

function ice_viscosity(T::Float64)
    """
    Arguments:
    T::Float64 --- A temperature value in (Kelvin) of float64

    Returns:
    ice_vis -- The ice viscosity in (Pa*s) that is dependent on temperature argument value

    Info:
    The equation used is a Newtonian formulation for temperature-dependent ice viscosity from Nimmo(2004)
    """
    Q = 40000.0 # Activation Enegry (J/mol)
    R_cont = 8.314 # Gas Constant (J/mol*K)
    ice_vis = (1e15)*exp((Q*(273.0-T))/(R_cont*(273.0*T)))
    upperlimit = 1e25
    lowerlimit = 1e12
    if ice_vis < lowerlimit
        ice_vis = lowerlimit
    elseif ice_vis > upperlimit
        ice_vis = upperlimit
    else
        ice_vis = ice_vis
    end
    return ice_vis
end

ice_viscosity (generic function with 1 method)

# Material Setup

In [10]:
struct Materials
    """
    Building composite types with a list of fields (or attributes) which are names that will be used as variables to store data for
    objects of the new types.

    Fields:
    alpha::Vector{Float64} -- A vector(1-D Array) that holds values of Thermal expansion in (1/K)
    rho0::Vector{Float64} -- A vector (1-D Array) that holds values of  Density in (kg/m^3)
    Hr::Vector{Float64} -- A vector(1-D Array) that holds values of Radiogenic heat production in (W/m^3)
    Cp::Vector{Float64} -- A vector(1-D Array) that holds values of Heat capacity in (J/kg*K)
    kThermal::Vector{Float64} -- A vector(1-D Array) that holds values of Thermal conductivity in (W/m*K)
    eta::Vector{Float64} -- A vector(1-D Array) that holds values of Viscosity in (Pa*s)

    Info:
    Each vector hold 3 elements which correspond to layer
    # 1 - subsurface global ocean
    # 2 - icy shell
    # 3 - sticky air
    """
    alpha::Vector{Float64} # Thermal expansion (1/K)
    rho0::Vector{Float64} # Density (kg/m^3)
    Hr::Vector{Float64} # Radiogenic heat production (W/m^3)
    Cp::Vector{Float64} # Heat capacity (J/kg*K)
    kThermal::Vector{Float64} # Thermal conductivity (W/m*K)
    eta::Vector{Float64} # Viscosity (Pa*s)
    function Materials()
        new([0.0,0.0,0.0],[1000.0,920.0,1.0],[0.0,0.0,0.0],[4180.0,2100.0,1.0e6],[0.5610,2.1,0.024],[1e12,1e15,1e17])
    end
end

function update_marker_prop!(markers::Markers,materials::Materials)
    eta = markers.scalarFields["eta"]
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
        if mmat[i] == 2
            markers.scalars[eta,i] = ice_viscosity(markers.scalars[T,i])
        end
    end
end

function update_marker_temp!(markers::Markers,materials::Materials)
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        if mmat[i] == 1
            markers.scalars[T,i] = 273.0
        elseif mmat[i] == 3
            markers.scalars[T,i] = 100.0
        end
    end
end

# function update_marker_viscosity!(markers::Markers,materials::Materials)
#     eta = markers.scalarFields["eta"]
#     T = markers.scalarFields["T"]
#     mmat = markers.integers[markers.integerFields["material"],:]
#     for i in 1:markers.nmark
#         if mmat[i] == 2
#             markers.scalars[eta,i] = ice_viscosity( markers.scalars[T,i] )
#         end
#     end
# end


# function update_marker_density!(markers::Markers,materials::Materials)
#     rho = markers.scalarFields["rho"]
#     T = markers.scalarFields["T"]
#     mmat = markers.integers[markers.integerFields["material"],:]
#     for i in 1:markers.nmark
#         markers.scalars[rho,i] = materials.rho0[mmat[i]]
#     end
# end


function initial_conditions!(markers::Markers,materials::Materials)
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    Cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    kThermal = markers.scalarFields["kThermal"]
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        hice = initial_ice_depth(mx)
        hsurf = initial_surface_depth(mx)
        if my > hice
            # subsurface global ocean
            markers.integers[material,i] = 1
            markers.scalars[T,i] = 273.0
            markers.scalars[eta,i] = materials.eta[1]
            markers.scalars[alpha,i] = materials.alpha[1]
            markers.scalars[Cp,i] = materials.Cp[1]
            markers.scalars[Hr,i] = materials.Hr[1]
            markers.scalars[kThermal,i] = materials.kThermal[1]
        elseif my > hsurf
            # icy shell
            markers.integers[material,i] = 2
            markers.scalars[T,i] = 100.0+((273.0-100.0)/(hice-hsurf))*(my-hsurf)
            # Viscosity is being assigned by the function ice_viscosity and being updated by the function update_markers_prop!
            markers.scalars[alpha,i] = materials.alpha[2]
            markers.scalars[Cp,i] = materials.Cp[2]
            markers.scalars[Hr,i] = materials.Hr[2]
            markers.scalars[kThermal,i] = materials.kThermal[2]
        else
            # sticky air
            markers.integers[material,i] = 3
            markers.scalars[T,i] = 100.0
            markers.scalars[eta,i] = materials.eta[3]
            markers.scalars[alpha,i] = materials.alpha[3]
            markers.scalars[Cp,i] = materials.Cp[3]
            markers.scalars[Hr,i] = materials.Hr[3]
            markers.scalars[kThermal,i] = materials.kThermal[3]
        end
    end
    update_marker_prop!(markers,materials)
end

function get_i_interface(grid::CartesianGrid,mat,contour_value)
    # Setting up inital model schematic
    i_interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if i_mat[i,j] == contour_value
                i_interface_position[j] = grid.yc[j]
                break
            elseif i_mat[i+1,j] < contour_value
                # interface is located within this cell.
                i_interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(i_mat[i+1,j]-i_mat[i,j])*(contour_value-i_mat[i,j])
                break
            end
            i = i+1
        end
    end
    return i_interface_position
end

function get_interface(grid::CartesianGrid,mat,contour_value)
    # Finding interfaces
    interface_position = zeros(Float64,grid.nx+1);
        for j in 1:grid.nx+1
        i = 1
            while i <= grid.ny
                if mat[i,j] == contour_value
                    interface_position[j] = grid.yc[i]
                    break
                elseif mat[i+1,j] < contour_value
                    # interface is located within this cell.
                    interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j])*(contour_value-mat[i,j])
                    break
                end
                i = i+1
            end
        end
    return interface_position
end

# function update_interface_temp!(markers::Markers,materials::Materials)
#     T = markers.scalarFields["T"]
#     mmat = markers.integers[markers.integerFields["material"],:]
#     ocean_ice_interface = get_interface(grid,mat,1.5)
#     air_ice_interface = get_interface(grid,mat,2.5)
#     for i in 1:markers.nmark
#         if mmat[i] ==
#         markers.scalars[T,i] = 273.0
#         elseif mmat[i] == air_ice_interface
#         markers.scalars[T,i] = 100.0
#         end
#     end
# end

function get_hydrostatic_pressure(grid::CartesianGrid,rho_vy::Matrix{Float64},gy::Float64)
    hydro_press = zeros(Float64,(grid.ny+1,grid.nx))    
    for j in 1:grid.nx
        for i in 1:grid.ny+1
            if i == 1
                hydro_press[i,j] = 0.0
            else
               hydro_press[i,j] = (hydro_press[i,j]+rho_vy[i-1,j]).*(gy*(grid.yc[i]-grid.yc[i-1]))
            end
        end 
    end 
    return hydro_press
end

get_hydrostatic_pressure (generic function with 1 method)

# Model Setup

In [ ]:
# nstep=10
nx = 91
ny = 101
W = 90000.0
H = 90000.0
gx = 0.0
gy = 0.113

# Tbctype = [-1,-1,1,1] #left, right, top, bottom
Tbctype = [1,-1,1,1]
# Tbcval = [0.0,0.0,100.0,273.0] #left, right, top, bottom
Tbcval = [273.0,0.0,100.0,273.0]
bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.
materials = Materials()

markx = 6
marky = 6
seconds_in_year = 3.15e7
plot_interval = 1e6*seconds_in_year # plot interval in seconds
end_time = 3e7*seconds_in_year
dtmax = plot_interval
grid = CartesianGrid(W,H,nx,ny)
println("Creating Markers...")
@time markers = Markers(grid,["alpha","T","rho","eta","Cp","Hr","kThermal"],["material"] ; nmx=markx,nmy=marky,random=true)
println("Initial condition...")
@time initial_conditions!(markers, materials)

time_plot = []
max_topo = []
### Setting up agruments for interface function ###
# initial
i_air_ice_interface = zeros(Float64,grid.nx+1)
i_ocean_ice_interface = zeros(Float64,grid.nx+1)
i_mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
i_air_ice_interface = get_i_interface(grid,i_mat,2.5)
i_ocean_ice_interface = get_i_interface(grid,i_mat,1.5)

# after
air_ice_interface = zeros(Float64,grid.nx+1)
ocean_ice_interface = zeros(Float64,grid.nx+1)
mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")


### Setting up agruments for termination criteria ###
max_step::Int64=0
max_time::Float64=-1.0
max_time::Float64 = max_time == -1.0 ? typemax(Float64) : max_time
max_step::Int64 = max_step == -1 ? typemax(Int64) : max_step

# # define arrays for k, rho, cp, H at the basic nodes. Fill them with constant values for now.
# kThermal = 3.0 .*ones(grid.ny,grid.nx);

time = 0.0
iout= 0
last_plot = 0.0
dt = 1e10

rho_c = nothing
rho_vx = nothing
rho_vy = nothing
alpha = nothing
Hr = nothing
Cp_c = nothing
eta_s = nothing
eta_n = nothing
vxc = nothing
vyc = nothing
T = nothing
dTmax = nothing
dTemp = nothing
Tnew = nothing
Tlast = nothing
x_time = nothing
kThermal = nothing

itime = 1
output_dir = "test"

terminate = false
while !terminate
    # 0. update the markers properties
    update_marker_prop!(markers,materials)
    update_marker_temp!(markers,materials)
#    update_interface_temp!(markers,materials)
    # 1. Transfer properties markers -> nodes
    # 1a. Basic Nodes
    eta_s_new, = marker_to_stag(markers,grid,["eta",],"basic")
    # 1b. Cell Centers
    rho_c_new,Cp_c_new,alpha_new,eta_n_new,Tlast_new,Hr_new,kThermal_new = marker_to_stag(markers,grid,["rho","Cp","alpha","eta","T","Hr","kThermal"],"center")
    # 1c. Vx and Vy nodes:
    rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx")
    rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy")

    # deal with any NaN values from interpolation:
    if itime > 1
        if any(isnan.(eta_s_new))
            println("found nan values")
        end
        replace_nan!(eta_s,eta_s_new)
        replace_nan!(rho_c,rho_c_new)
        replace_nan!(Hr,Hr_new)
        replace_nan!(Cp_c,Cp_c_new)
        replace_nan!(alpha,alpha_new)
        replace_nan!(eta_n,eta_n_new)
        replace_nan!(Tlast,Tlast_new)
        replace_nan!(rho_vx,rho_vx_new)
        replace_nan!(rho_vy,rho_vy_new)
        replace_nan!(kThermal,kThermal_new)
    end
    # Copy field data
    kThermal = copy(kThermal_new)
    rho_vx = copy(rho_vx_new)
    rho_vy = copy(rho_vy_new)
    rho_c = copy(rho_c_new)
    Hr = copy(Hr_new)
    Cp_c = copy(Cp_c_new)
    alpha = copy(alpha_new)
    eta_s = copy(eta_s_new)
    eta_n = copy(eta_n_new)
    Tlast = copy(Tlast_new)

    if itime == 1
        ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
        cell_center_to_markers!(markers,grid,Tlast,markers.scalars[[markers.scalarFields["T"],],:])
    else
        ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
    end

    # 2. Assemble and solve the stokes equations
    L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy;dt=dt)
    stokes_solution = L\R
    vx,vy,P = unpack(stokes_solution,grid;ghost=true)

    # Get the velocity at the cell centers:
    vxc,vyc = velocity_to_centers(grid,vx,vy)
    adiabatic_heating = compute_adiabatic_heating(grid,rho_c,Tlast,alpha,gx,gy,vxc,vyc)*0.0
    shear_heating = compute_shear_heating(grid,vx,vy,eta_n,eta_s)*0.0
    H = (adiabatic_heating .+ shear_heating .+ Hr).*0.0

    # 3. Compute the advection timestep
    if itime > 1
        this_dtmax = min(1.2*dt,dtmax)
    else
        this_dtmax = dtmax
    end
    dt = compute_timestep(grid,vxc,vyc;dtmax=this_dtmax,cfl=0.25)
    diffusion_timestep = (grid.x[2]-grid.x[1])^2 / 1e-6
    if dt > diffusion_timestep
        dt = diffusion_timestep
    end

    dTmax = Inf
    dTemp = nothing
    Tnew = nothing
    titer = 1
    for titer=1:2
        # assemble and solve the energy equation
#         println("Trying with timestep ",dt/3.15e7/1e6," Myr")
        println("Trying with timestep ",dt/seconds_in_year/1e3," kyr")
#         println("Trying with timestep ",dt/seconds_in_year," yr")
#         println("Trying with timestep ",dt," Seconds")
        L,R = assemble_energy_equation_center(grid,rho_c,Cp_c,kThermal,H,Tlast,dt,Tbctype,Tbcval)
        Tnew = L\R;
        Tnew = reshape(Tnew,grid.ny,grid.nx);
        Tnew = ghost_temperature_center(grid,Tnew,Tbctype,Tbcval)
        T = copy(Tnew)

        dTemp = Tnew-Tlast
        # compute the maximum temperature change
        dTmax = maximum(abs.(dTemp[2:end-1,2:end-1]))
#         println("dTmax=",dTmax," dt=",dt/3.15e7/1e6," Myr")
        println("dTmax=",dTmax," dt=",dt/seconds_in_year/1e3," kyr")
#         println("dTmax=",dTmax," dt=",dt/seconds_in_year," Seconds")
        dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)
        if dTmax < 10.0
            break
        end
    end

    dT_subgrid_node = subgrid_temperature_relaxation_center!(markers,grid,Tlast,Cp_c[1,1],kThermal[1,1],dt)
    #dT_remaining = dTemp - dT_subgrid_node
    Tnew = Tnew .- dT_subgrid_node

    cell_center_change_to_markers!(markers,grid,Tnew,"T")

    # Checking Termination Criteria, time is in Myr
    if time >= max_time || itime >= max_step
        terminate = true
    end

#     if time == 0.0 || time - last_plot >= plot_interval  || mod(itime,100) == 0
#         last_plot = time
#         name = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
#         vn = velocity_to_basic_nodes(grid,vxc,vyc)
#         visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name)
#         name = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
#         visualization(markers,time/seconds_in_year;filename=name)
#         iout += 1
#     end

    if time == 0.0 || mod(itime,100) == 0 || true
        last_plot = time
        # Gird output
        name1 = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
        println("Writing visualization file = ",name1)
        vn = velocity_to_basic_nodes(grid,vxc,vyc)
        visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name1)
        # Markers output
        name2 = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
        println("Writing visualization file = ",name2)
        visualization(markers,time/seconds_in_year;filename=name2)
        iout += 1
    end

    # println("Min/Max velocity: ",minimum(vyc)," ",maximum(vyc))
    # Moving the markers and advancing to the next timestep
    move_markers_rk4!(markers,grid,vx,vy,dt,continuity_weight=1/3)
    time += dt
    itime += 1
    # println("Finished Step ",itime," time=",time/seconds_in_year/1e6," Myr")
    println("Finished Step ",itime," time=",time/seconds_in_year/1e3," kyr")
#     println("Finished Step ",itime," time=",time/seconds_in_year," yr")
#     println("Finished Step ",itime," time=",time," Seconds")
    x_time = @sprintf("%.3g",time/3.15e7/1e6);
    mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
    ocean_ice_interface = get_interface(grid,mat,1.5)
    air_ice_interface = get_interface(grid,mat,2.5)
    append!(time_plot,time)
    append!(max_topo,maximum(ocean_ice_interface.-(0.5*90000.0)))
end

figure()
scatter(markers.x[1,:],markers.x[2,:],c="b",s=0.1)
show()

Creating Markers...
  0.008627 seconds (21 allocations: 33.372 MiB)
Initial condition...
  0.088412 seconds (1.55 M allocations: 24.339 MiB, 11.68% gc time)
assigning left boundary temperature 273.0
Trying with timestep 0.2926226545373838 kyr
assigning left boundary temperature 273.0
dTmax=3.15515665384099 dt=0.2926226545373838 kyr


# Plot of Hydrostatic Pressure

In [ ]:
hydrostatic_pressure = get_hydrostatic_pressure(grid,rho_vy,gy)
display(hydrostatic_pressure)

In [1]:
hydrostatic_pressure = get_hydrostatic_pressure(grid,rho_vy,gy)
figure()
plot(hydrostatic_pressure[:,1],grid.yc)
gca().invert_yaxis()
#gca().set_aspect("equal")
gca().set_ylabel(L"Depth\,(m)")
gca().set_xlabel(L"Hydrostatic\,Pressure\,(Pa)")
show()

LoadError: UndefVarError: get_hydrostatic_pressure not defined

# 1-D Test

In [ ]:
function oneD_hydrostatic_pressure(depth::Array,density::Array,gravity::Float64)
    """
    Arguments:
    depth -- the depth in (m)
    density -- the density of the fluid in (kg/^3)
    gravity -- the acceleration due to gravity in (m/s^2)

    Returns:
    pressure -- the hydostatic pressure in (MPa)
    """
    pressure = zeros(1,length(depth))
    for i in 1:length(depth)
        hice = initial_ice_depth(1.0)/1000
        hsurf = initial_surface_depth(1.0)/1000
        if i > hice
            pressure[i] = density[1]*gravity*(depth[i]-hsurf)*1000.0
        elseif i > hsurf
            pressure[i] = density[2]*gravity*(depth[i]-hsurf)*1000.0
        else
            pressure[i] = 0.0
        end
    end
    return pressure/1e+6
end

depth = collect(1:90)
density = [1000.0,920.0]
pressure = hydrostatic_pressure(depth,density,0.113)

for (depth,pressure) in zip(depth,pressure)
    using Printf
    println("The hydrostatic pressure at a depth of $(depth)(km) is $(@sprintf("%.3f",pressure))(MPa)")
end

In [ ]:
max_p = maximum(markers.scalars[markers.scalarFields["ph"],:])

In [ ]:
py"""
import numpy as np
import seafreeze.seafreeze as sf
import matplotlib.pyplot as plt

P = np.arange(0, 1300, 5.0)
T = np.arange(200, 300, 1.0)
PT = np.array([P, T], dtype=object)
out = sf.whichphase(PT)
#print(out)

TT, PP = np.meshgrid(T, P)
fig, ax = plt.subplots()
for i in PT:
    for p in sf.phasenum2phase.keys():
        pi = out==p
        plt.scatter(TT[pi], PP[pi], label=sf.phasenum2phase[p])

plt.legend()
plt.xlabel('temperature (K)')
plt.ylabel('pressure (MPa)')
plt.title('$H_{2}O$ phase diagram')
"""


In [ ]:
gy

In [ ]:
rho_vy